In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from collections import Counter
import collections
import random
from six.moves import urllib
from six.moves import xrange  # pylint: disable=redefined-builtin
import Bio
from Bio import SeqIO
import os
import concurrent.futures
import functools
from functools import partial
import math
import threading
import time
import random
from random import shuffle
import pickle
import tempfile

# k-mer size to use
k = 9

#
# NOTE!!!!!!!!!!!!!!!!
#
# We can reduce problem space if we get the reverse complement, and add a bit to indicate reversed or not...
# Not really.... revcomp just doubles it back up again....
#
# Also -- Build a recurrent network to predict sequences that come after a given kmer?
# Look at word2vec, dna2vec, bag of words, skip-gram
#

# Problem space
space = 5 ** k

def partition(n, step, coll):
    for i in range(0, len(coll), step):
        if (i+n > len(coll)):
            break #  raise StopIteration...
        yield coll[i:i+n]
        
def get_kmers(k):
    return lambda sequence: partition(k, k, sequence)

def convert_nt(c):
    return {"N": 0, "A": 1, "C": 2, "T": 3, "G": 4}.get(c, 0)

def convert_nt_complement(c):
    return {"N": 0, "A": 3, "C": 4, "T": 1, "G": 2}.get(c, 0)

def convert_kmer_to_int(kmer):
    return int(''.join(str(x) for x in (map(convert_nt, kmer))), 5)

def convert_kmer_to_int_complement(kmer):
    return int(''.join(str(x) for x in reversed(list(map(convert_nt_complement, kmer)))), 5)

def convert_base5(n):
    return {"0": "N", "1": "A", "2": "C", "3": "T", "4": "G"}.get(n,"N")

def convert_to_kmer(kmer):
    return ''.join(map(convert_base5, str(np.base_repr(kmer, 5))))

# Not using sparse tensors anymore.
   
tf.logging.set_verbosity(tf.logging.INFO)

# Get all kmers, in order, with a sliding window of k (but sliding 1bp for each iteration up to k)
# Also get RC for all....

def kmer_processor(seq,offset):
    return list(map(convert_kmer_to_int, get_kmers(k)(seq[offset:])))

def get_kmers_from_seq(sequence):
    kmers_from_seq = list()

    kp = functools.partial(kmer_processor, sequence)
    
    for i in map(kp, range(0,k)):
        kmers_from_seq.append(i)

    rev = sequence[::-1]
    kpr = functools.partial(kmer_processor, rev)
    
    for i in map(kpr, range(0,k)):
        kmers_from_seq.append(i)
            
#    for i in range(0,k):
#        kmers_from_seq.append(kmer_processor(sequence,i))
#    for i in range(0,k):
#        kmers_from_seq.append(kmer_processor(rev, i))
    return kmers_from_seq

data = list()

def load_fasta(filename):
    data = dict()
    for seq_record in SeqIO.parse(filename, "fasta"):
        data.update({seq_record.id:
                     get_kmers_from_seq(seq_record.seq.upper())})
    return(data)
        
def get_kmers_from_file(filename):
    kmer_list = list()
    for seq_record in SeqIO.parse(filename, "fasta"):
        kmer_list.extend(get_kmers_from_seq(seq_record.seq.upper()))
    return set([item for sublist in kmer_list for item in sublist])

all_kmers = set()

# Very slow, should make this part concurrent...

def find_all_kmers(directory):
    kmer_master_list = list()
    files = [directory + "/" + f for f in os.listdir(directory)]
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        for i in executor.map(get_kmers_from_file, files):
            kmer_master_list.extend(list(i))
            kmer_master_list = list(set(kmer_master_list))
            print("Total unique kmers: " + str(len(set(kmer_master_list))))
    return set(kmer_master_list)

def get_categories(directory):
    data = list()
    files = os.listdir(directory)
    for filename in files:
        for seq_record in SeqIO.parse(directory + "/" + filename, "fasta"):
            data.append(seq_record.id)
    data = sorted(list(set(data)))
    return(data)

In [2]:
# filegen = training_file_generator("training-files/")

# training_data = load_fasta(filegen())


In [3]:
replicons_list = get_categories("training-files/")


In [4]:
# Because this was run at work on a smaller sample of files....
# with open("all_kmers_subset.txt", "w") as f:
#     for s in all_kmers:
#         f.write(str(s) +"\n")

# Because this was run at work on a smaller sample of files....
all_kmers = list()
# with open("all_kmers_subset.txt", "r") as f:
#     for line in f:
#         all_kmers.append(int(line.strip()))

all_kmers = pickle.load(open("all_kmers.p", "rb"))

all_kmers = set(all_kmers)
len(all_kmers)

269132

In [5]:
# len(data)

# all_kmers = set([item for sublist in data for item in sublist])
unused_kmers = set(range(0, space)) - all_kmers

kmer_dict = dict()
reverse_kmer_dict = dict();

a = 0
for i in all_kmers:
    kmer_dict[i] = a
    reverse_kmer_dict[a] = i
    a += 1
    
kmer_count = len(all_kmers)

[len(all_kmers), len(unused_kmers), space]

[269132, 1683993, 1953125]

In [104]:
def training_file_generator(directory):
    files = [directory + "/" + f for f in os.listdir(directory)]
    random.shuffle(files)
    def gen():
        nonlocal files
        if (len(files) == 0):
            files = [directory + "/" + f for f in os.listdir(directory)]
            random.shuffle(files)
        return(files.pop())
    return gen

def gen_random_training_data(input_data, window_size):
    rname = random.choice(list(input_data.keys()))
    rdata = random.choice(input_data[rname])
    idx = random.randrange(window_size + 1, len(rdata) - window_size - 1)
    tdata = list();
    for i in range(idx - window_size - 1, idx + window_size):
        if (i < 0): continue
        if (i >= len(rdata)): break
        if type(rdata[idx]) == list: break;
        if type(rdata[i]) == list: break
        tdata.append(kmer_dict[rdata[i]])
    return tdata, rname

# The current state is, each training batch is from a single FASTA file (strain, usually)
# This can be ok, as long as training batch is a large number
# Need to speed up reading of FASTA files though, maybe pyfaidx or something?

def generate_training_batch(data, batch_size, window_size):
    training_batch_data = list();
    while len(training_batch_data) < batch_size:
         training_batch_data.append(gen_random_training_data(data, 
                                                             window_size))
    return training_batch_data

def train_input_fn():
    rdata = generate_training_batch(training_data, 1, window_size)[0]
    return {"kmer_input": tf.nn.embedding_lookup(embeddings, rdata[0])}, {"Replicon": rdata[1]}
        

In [105]:
# filegen = training_file_generator("training-files/")

# training_data = load_fasta(filegen())
# training_data_backup = training_data

# len(training_data)

# gen_random_training_data(training_data, 7)

# generate_training_batch(training_data, 5, 7)

# random.choice(list(input_data.keys()))

train_input_fn()

({'kmer_input': <tf.Tensor 'embedding_lookup_4:0' shape=(15, 128) dtype=float32>},
 {'Replicon': 'pSymA'})

In [106]:
filegen = training_file_generator("training-files/")

# training_data = load_fasta(filegen())

batch_size = 1024
embedding_size = 128
window_size = 7

validation_set = generate_training_batch(training_data, 10000, window_size)
# validation_kmers = list(set([i[0] for i in validation_set]))
# del validation_set

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 1024
valid_examples = validation_set
num_sampled = 256
del validation_set

In [107]:
replicons.input_layer("Main")

AttributeError: '_VocabularyListCategoricalColumn' object has no attribute 'input_layer'

In [111]:
tf.one_hot(replicons, 1)

TypeError: Failed to convert object of type <class 'tensorflow.python.feature_column.feature_column._VocabularyListCategoricalColumn'> to Tensor. Contents: _VocabularyListCategoricalColumn(key='Replicon', vocabulary_list=('Emel_M2_Main', 'Emel_M2_Uni1', 'Emel_M2_Uni2', 'Emel_M2_Uni3', 'Emel_WSM419_Main', 'Emel_WSM419_Uni1', 'Emel_WSM419_Uni11', 'Emel_WSM419_Uni2', 'HM006_Accessory_A', 'KH35c_Accessory_A', 'M162_Accessory_A', 'M270_Accessory_A', 'M270_Accessory_B', 'M270_Accessory_C', 'Main', 'Rm41_Accessory_A', 'T073_Accessory_A', 'USDA1021_Accessory_A', 'USDA1157_Accessory_A', 'pHRB800', 'pHRC017', 'pRmeGR4a', 'pRmeGR4b', 'pSINME01', 'pSINME02', 'pSMED03_WSM419', 'pSymA', 'pSymB'), dtype=tf.string, default_value=-1, num_oov_buckets=0). Consider casting elements to a supported type.

In [108]:
graph = tf.Graph()

with graph.as_default():
    
  # Load embedding
  kmers = tf.Variable(tf.constant(0.0, shape=[kmer_count, 128]),
                     trainable=False, name="kmers")

  embeddings = np.load("embeddings_200000.npy")

  embedding_placeholder = tf.placeholder(tf.float32, [kmer_count, 128])
  embedding_init = kmers.assign(embeddings)

  # Input data.
  # Take 1 kmer and the 7 on each side of it
  # So for k=9, we are testing 135bp
  # So n = 15
  train_input = tf.placeholder(tf.int32, shape=[15]) 
  train_label = tf.placeholder(tf.int32, shape=[1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
  
  kmer_input = tf.nn.embedding_lookup(embeddings, train_input)
    
  replicons = tf.feature_column.categorical_column_with_vocabulary_list(
    "Replicon", get_categories("training-files/"))

  model_dir = tempfile.mkdtemp()

#  m = tf.estimator.DNNLinearCombinedClassifier(
#           model_dir=model_dir,
#           n_classes=len(replicons_list),
#           dnn_feature_columns=[kmers],
#           dnn_hidden_units=[100,50])

  m = tf.estimator.DNNClassifier(
            feature_columns=[kmer_input, replicons],
            hidden_units=[500,50],
            n_classes=len(replicons_list),
            model_dir=model_dir)
    
  m.train(input_fn=train_input_fn, steps=1)

  # Add variable initializer.
#   init = tf.global_variables_initializer()
#   saver = tf.train.Saver()

TypeError: Expected int32, got 'pSymB' of type 'str' instead.

In [98]:
tf.nn.embedding_lookup(embeddings, train_input)

<tf.Tensor 'embedding_lookup_1:0' shape=(15, 128) dtype=float32>

In [125]:
tf.feature_column.indicator_column(replicons)

_IndicatorColumn(categorical_column=_VocabularyListCategoricalColumn(key='Replicon', vocabulary_list=('Emel_M2_Main', 'Emel_M2_Uni1', 'Emel_M2_Uni2', 'Emel_M2_Uni3', 'Emel_WSM419_Main', 'Emel_WSM419_Uni1', 'Emel_WSM419_Uni11', 'Emel_WSM419_Uni2', 'HM006_Accessory_A', 'KH35c_Accessory_A', 'M162_Accessory_A', 'M270_Accessory_A', 'M270_Accessory_B', 'M270_Accessory_C', 'Main', 'Rm41_Accessory_A', 'T073_Accessory_A', 'USDA1021_Accessory_A', 'USDA1157_Accessory_A', 'pHRB800', 'pHRC017', 'pRmeGR4a', 'pRmeGR4b', 'pSINME01', 'pSINME02', 'pSMED03_WSM419', 'pSymA', 'pSymB'), dtype=tf.string, default_value=-1, num_oov_buckets=0))